In [0]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *


In [0]:
SparkSession.builder.appName("book rating").getOrCreate()

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
rawDF= spark.read.option("header",True).option("inferSchema", True).csv("/FileStore/tables/book_ratings.csv")

In [0]:
from pyspark.ml.recommendation import ALS

In [0]:
als = ALS(userCol="userId",itemCol="movieId",ratingCol="rating",coldStartStrategy="drop",nonnegative=True) #null kayıtları almamak için coldStartStrategy kullandık

In [0]:
trainDF,testDF= rawDF.randomSplit([0.7,0.3])  #model oluşturduktan sonra gerçek verinin ve tahminin değeri için iki df oluşturduk.

In [0]:
trainDF.count() #test data ve train datada kaç veri olduğunu gördük. train data %70 test data %30 olduğu için yani yukarıda verdiğimiz değerler. train data verileri daha çok oldu


Out[27]: 70310

In [0]:
testDF.count()

Out[28]: 29694

In [0]:
model = als.fit(trainDF)  #trainDF deki verileri fit edip yani sıkıştırıp model adlı df ye ekledik.

In [0]:
predictDF = model.transform(testDF)  #testDF deki verileri tahmin ettik 

In [0]:
predictDF.show() #yani burada yapmış olduğumuz işlem rating puanlarını tahmin  etmek oldu bunun için de iki tane veri seti oluşturduk trainDF ve testDF adında 70 ve 30 olarak oaranladık bu oranları değiştirebiliriz. biz değiştirdikçe tahmin değerleri değişir eğer train data değerini yüksek girersek tahmin yapma seviyemiz artar.

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|   463|   1088|   3.0|1050499697| 3.1143305|
|   463|   3175|   3.0|1049913808|  3.579974|
|   471|  44022|   3.5|1239653591| 3.4372096|
|   243|   1645|   3.0|1094220341| 3.2757053|
|   243|   2122|   2.0|1094227385| 2.6676304|
|   137|   1580|   5.0| 946411685| 3.5466301|
|   251|   1580|   5.0|1476550351|  4.259323|
|   580|   6620|   4.0|1155617916| 3.3240144|
|    65|   1238|   2.0| 945150351| 3.8526356|
|   472|   2366|   5.0| 994737953|  4.496359|
|   472|   6620|   4.0|1156728234| 4.2714853|
|   472|  44022|   3.5|1164753301| 2.4663448|
|   597|   1238|   4.0| 940708124|  4.498716|
|   597|   1580|   5.0| 940707179| 3.9730666|
|   155|   1580|   5.0| 943350292|  3.114944|
|    34|   2366|   4.0| 973747877|  4.372559|
|   497|   1959|   5.0| 940364289| 4.2263446|
|   596|   1580|   3.0|1138658670| 3.4303463|
|   412|    833|   1.0| 993091607|

In [0]:
from pyspark.ml.evaluation  import RegressionEvaluator  #hata payını bulmak için import ettik

In [0]:
regeva = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")

In [0]:
reseCal=regeva.evaluate(predictDF)

In [0]:
print(reseCal)  #veri setimiz yani test datamız 0,92 oranında doğru 1 e yaklaştıkça doğruluk artar.

0.9231367715867134
